In [4]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)
     |████████████████████████████████| 112kB 5.0MB/s 


In [5]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score,KFold
import xgboost as xgb

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [6]:
cd  "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_2/dw_matrix_car


In [7]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorize_value = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorize_value
  else:
     df[feat + SUFFIX_CAT] = factorize_value

In [0]:
df =df[ df['price_currency'] != 'EUR']

In [9]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y,cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## Decision Tree

In [15]:
run_model(DecisionTreeRegressor(max_depth= 5), cat_feats)

(-19566.588937368328, 90.61814865166)

## Radom Forest

In [16]:
model = RandomForestRegressor(max_depth= 5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18732.32439658373, 112.97342773034612)

## XGBoost

In [20]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_reate': 0.1,
    'seed': 0
   }

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[17:36:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:37:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13044.900535034605, 102.060458296808)

In [21]:
m =  xgb.XGBRegressor(max_depth= 5, n_estimators=50, learning_rate=0.1,seed =0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names = cat_feats )

[17:40:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1224 ± 0.0019,param_napęd__cat
0.1185 ± 0.0028,param_rok-produkcji__cat
0.1115 ± 0.0013,param_stan__cat
0.0610 ± 0.0018,param_skrzynia-biegów__cat
0.0523 ± 0.0018,param_faktura-vat__cat
0.0464 ± 0.0015,param_moc__cat
0.0285 ± 0.0009,param_marka-pojazdu__cat
0.0241 ± 0.0004,param_typ__cat
0.0225 ± 0.0007,feature_kamera-cofania__cat
0.0193 ± 0.0006,param_pojemność-skokowa__cat


In [0]:
feats =[
'param_napęd__cat','param_rok-produkcji__cat','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc__cat','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_światła-led__cat']

In [0]:
len(feats)

In [25]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[17:51:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:51:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:51:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13208.498613410682, 47.72442373981382)

In [0]:
df['param_rok-produkcji'] =df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)

In [0]:
feats =[
'param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_światła-led__cat']

In [31]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[17:57:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:57:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:57:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11159.683286420848, 33.719233628580156)

In [0]:

df['param_moc'] =df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [0]:
feats =[
'param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_światła-led__cat']

In [39]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:04:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:04:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:04:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9585.893931475613, 58.83501929545337)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '')))


In [47]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:09:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:09:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:10:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9442.766736097698, 96.95757061896506)